In [2]:
using CairoMakie, Dierckx, QuantEcon, LinearAlgebra, Optim

In [10]:
pars = (;r = 0.04, # interest rate
    β = 0.9, # discount factor
    γ = 1.0, # risk aversion
    nz = 19, # number of grid points for z
    ρ = 0.9, # persistence of AR1
    μ = 0.0, # mean of AR1
    σ = 0.003, # std of AR1
    na = 31, # number of asset grid points
    θ = 4.0, # grid expansion factor
    toler = 4e-7, # tolerance
    maxiter = 1000, # maximum no. of iterations
    lb = 0.0, # lower bound of capital grid
    ub = 100.0) # upper bound of capital grid

(r = 0.04, β = 0.9, γ = 1.0, nz = 19, ρ = 0.9, μ = 0.0, σ = 0.003, na = 31, θ = 4.0, toler = 4.0e-7, maxiter = 1000, lb = 0.0, ub = 100.0)

In [8]:
function utility(c, pars)
    (; γ) = pars
    if γ == 1.0
        return log(c)
    else
        return c^(1-γ)/(1-γ)
    end
end

function ar1(pars)
    (;ρ, μ, σ, nz) = pars
    mc = QuantEcon.rouwenhorst(nz, μ, ρ, σ)
    return mc.p, mc.state_values
end

function exp_grid(pars)
    (; na, θ, lb, ub) = pars
    grid = LinRange(0.0,1.0,na)
    exp_grid = lb .+ (ub - lb) .* grid.^θ
    return exp_grid
end

function resources(Avals, Zvals, j, i, pars)
    (; r) = pars
    return (1+r)*Avals[j] + 3.0 * exp(Zvals[i])
end

function interpV(Avals, v_slice, pars)
    interp_v = Spline1D(Avals, v_slice, k=3, bc="extrapolate")
    return interp_v
end

function optimise(Avals, Zvals, v_init, v_new, policy, Π, pars)
    (; β, na, nz, lb, ub) = pars
    for i in 1:nz
        expected_value = v_init * Π[i,:]
        interpolation = interpV(Avals, expected_value, pars)
        for j in 1:na
            obj(ap) = - (utility(resources(Avals, Zvals, j, i, pars) - ap, pars) + β * interpolation(ap))
            ub = resources(Avals, Zvals, j, i, pars)  
            res = optimize(obj, lb, ub)
            policy[j,i] = res.minimizer
            v_new[j,i] = -res.minimum
        end
    end
    return v_new, policy
end

optimise (generic function with 1 method)

In [16]:
function vfi(v_init, pars)
    (; maxiter, toler, nz, na) = pars
    Π, Zvals = ar1(pars)
    Avals = exp_grid(pars)
    v_new = similar(v_init)
    policy = similar(v_init)
    error = toler + 1
    iter = 0
    if iter == 0
        println("Iterating...")
    end
    while ((error > toler) && (iter < maxiter))
        v_new, policy = optimise(Avals, Zvals, v_init, v_new, policy, Π, pars)
        error = maximum(abs.(v_new - v_init) ./ (1 .+ abs.(v_new)))
        v_init .= v_new
        if iter % 10 == 0
            println("--------------------")
            println("Iteration: $iter, Error: $error")
        end
        iter += 1
    end
    println("--------------------")
    println("Converged in $iter iterations")
    println("--------------------")
    return v_new, policy
end

v_init = ones(pars.na, pars.nz)

@time begin
v_out, pol_out = vfi(v_init, pars)
end

Iterating...
--------------------
Iteration: 0, Error: 1.0
--------------------
Iteration: 10, Error: 0.07232557610459449
--------------------
Iteration: 20, Error: 0.017830838228735674
--------------------
Iteration: 30, Error: 0.005632459966216119
--------------------
Iteration: 40, Error: 0.0019012546143692846
--------------------
Iteration: 50, Error: 0.0006556228447679164
--------------------
Iteration: 60, Error: 0.0002277264502175254
--------------------
Iteration: 70, Error: 7.929745145066869e-5
--------------------
Iteration: 80, Error: 2.763646541205109e-5
--------------------
Iteration: 90, Error: 9.634678814345413e-6
--------------------
Iteration: 100, Error: 3.359215060280822e-6
--------------------
Iteration: 110, Error: 1.17126280304498e-6
--------------------
Iteration: 120, Error: 4.083912833241455e-7
--------------------
Converged in 122 iterations
--------------------
  0.404196 seconds (271.72 k allocations: 36.708 MiB, 2.14% gc time, 33.53% compilation time)


([9.10012555176463 9.524389620476558 … 22.72837553860058 25.27592910178118; 9.102066449032996 9.525659880617907 … 22.728383528913824 25.275935179252116; … ; 22.80547060915417 22.80762456719113 … 27.23492503842318 29.61783108330198; 23.58319569564429 23.585165879297094 … 27.83779814118433 30.447238379442183], [4.0641029986558054e-16 1.0130567851419616e-15 … 73.56428292083669 115.87470022211228; 5.216321015191114e-16 2.6269781648927557e-16 … 73.5643964623109 115.87482130260845; … ; 74.66499555412081 74.69589626482069 … 158.1555864046015 210.9157386358425; 86.34113513617699 86.37207615433597 … 172.26669739779598 226.04246852237583])